https://github.com/dbouchabou/Fully-Convolutional-Network-Smart-Homes/tree/master

In [2]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = True)

Mounted at /content/drive


In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
cd "/content/drive/MyDrive/PAMAP2_Dataset"

/content/drive/MyDrive/PAMAP2_Dataset


In [4]:
data = pd.read_pickle('pamap2_for_lstm.pkl')

Take last

#### ~ 200k samples per id

In [5]:
columns_of_interest = ['time_stamp', 'activity_id','heart_rate', 'hand_temperature','hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y','hand_3D_acceleration_16_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y', 'hand_3D_gyroscope_z','chest_3D_acceleration_16_x', 'chest_3D_acceleration_16_y','chest_3D_acceleration_16_z','chest_3D_gyroscope_x', 'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z','ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y','ankle_3D_acceleration_16_z','ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z', 'id']

data = data[columns_of_interest]
data = data[data['id'] != 109]

In [6]:
from sklearn.model_selection import train_test_split
grouped_data = data.groupby('id')
train_data = pd.DataFrame()
val_test_data = pd.DataFrame()

for _, group in grouped_data:
    sorted_group = group.sort_values(by='time_stamp', ascending=True)

    #train has first 170k rows
    train_group = sorted_group.iloc[:120000]
    val_test_group = sorted_group.iloc[120000:]
    train_group = train_group[::40]
    val_test_group = train_group[::40]

    train_data = pd.concat([train_data, train_group])
    val_test_data = pd.concat([val_test_data, val_test_group])

# Split val_test_data into validation and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.5, random_state=42)

In [ ]:
train_data.shape

(24000, 23)

In [7]:
def segment_activities(df):
    activitiesSeq = []

    ponentialIndex = df.activity_id.ne(df.activity_id.shift())

    ii = np.where(ponentialIndex == True)[0]

    for i,end in enumerate(ii):
        if i > 0 :

          dftmp = df[ii[i-1]:end]
          activitiesSeq.append(dftmp)
    return activitiesSeq

In [8]:
train_activitySequences = segment_activities(train_data)
val_activitySequences = segment_activities(val_data)
test_activitySequences = segment_activities(test_data)

In [9]:
len(val_activitySequences)

230

#### with my columns

In [10]:
def generate_sentence(df2):
    sentence = ""

    # Define the column names you want to extract values from
    columns_of_interest = ['time_stamp', 'activity_id','heart_rate', 'hand_temperature','hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y','hand_3D_acceleration_16_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y', 'hand_3D_gyroscope_z','chest_3D_acceleration_16_x', 'chest_3D_acceleration_16_y','chest_3D_acceleration_16_z','chest_3D_gyroscope_x', 'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z','ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y','ankle_3D_acceleration_16_z','ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z', 'id']

    # Iterate over the columns of interest
    for column in columns_of_interest:
        # Get the value for the current column
        value = df2[column].values[0]  # Directly access the single value in the column

        # Add column name and value to the sentence
        sentence += "{}{}".format(column, value)

        # Add a space if it's not the last column
        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [11]:
def sequencesToSentences(activitySequences):
	sentences = []
	label_sentences = []

	for i in range(len(activitySequences)):

		sentence = generate_sentence(activitySequences[i])

		sentences.append(sentence)
		label_sentences.append(activitySequences[i].activity_id.values[0])

	return sentences, label_sentences

In [12]:
train_sentences, train_label_sentences = sequencesToSentences(train_activitySequences)
val_sentences, val_label_sentences = sequencesToSentences(val_activitySequences)
test_sentences, test_label_sentences = sequencesToSentences(test_activitySequences)

#### use 1 tokenizer for all

In [13]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
combined_sentences = train_sentences + val_sentences + test_sentences
tokenizer.fit_on_texts(combined_sentences)

# Tokenize train, validation, and test sets
train_indexed_sentences = tokenizer.texts_to_sequences(train_sentences)
val_indexed_sentences = tokenizer.texts_to_sequences(val_sentences)
test_indexed_sentences = tokenizer.texts_to_sequences(test_sentences)

In [ ]:
tokenizer1 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer1.fit_on_texts(train_sentences)
word_index = tokenizer1.word_index
train_indexed_sentences = tokenizer1.texts_to_sequences(train_sentences)

tokenizer2 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer2.fit_on_texts(val_sentences)
word_index = tokenizer2.word_index
val_indexed_sentences = tokenizer2.texts_to_sequences(val_sentences)

tokenizer3 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer3.fit_on_texts(test_sentences)
word_index = tokenizer3.word_index
test_indexed_sentences = tokenizer3.texts_to_sequences(test_sentences)

In [ ]:
len(tokenizer2.word_index)

4529

In [14]:
def slidingWindow(sequence,winSize,step=1):

    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")

    numOfChunks = int(((len(sequence)-winSize)/step)+1)

    # Do the work
    if winSize > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i in range(0,numOfChunks*step,step):
            yield sequence[i:i+winSize]

In [15]:
X_train_windowed = []
Y_train_windowed = []
winSize = 50
step = 1

for i,s in enumerate(train_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_train_windowed.append(chunk)
		Y_train_windowed.append(train_label_sentences[i])

X_val_windowed = []
Y_val_windowed = []
winSize = 50
step = 1

for i,s in enumerate(val_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_val_windowed.append(chunk)
		Y_val_windowed.append(val_label_sentences[i])


X_test_windowed = []
Y_test_windowed = []
winSize = 50
step = 1

for i,s in enumerate(test_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_test_windowed.append(chunk)
		Y_test_windowed.append(test_label_sentences[i])

In [16]:
train_padded_windows = pad_sequences(X_train_windowed)
test_padded_windows = pad_sequences(X_test_windowed)
val_padded_windows = pad_sequences(X_val_windowed)

In [17]:
Y_train_windowed = np.array(Y_train_windowed)
Y_test_windowed = np.array(Y_test_windowed)
Y_val_windowed = np.array(Y_val_windowed)

np.save("{}_{}_tpadded_x.npy".format("lstm",winSize), train_padded_windows)
np.save("{}_{}_tpadded_y.npy".format("lstm",winSize), Y_train_windowed)
np.save("{}_{}_ttpadded_x.npy".format("lstm",winSize), test_padded_windows)
np.save("{}_{}_ttpadded_y.npy".format("lstm",winSize), Y_test_windowed)
np.save("{}_{}_vpadded_x.npy".format("lstm",winSize), val_padded_windows)
np.save("{}_{}_vpadded_y.npy".format("lstm",winSize), Y_val_windowed)

#### Split on subjects:
  subjects 105 for val
  106 for test

In [18]:
all_data = pd.concat(activitySequences)

NameError: name 'activitySequences' is not defined

In [ ]:
unique_subject_ids = all_data['id'].unique()

train_data = []
val_data = []
test_data = []

for subject_id in unique_subject_ids:
    subject_data = all_data[all_data['id'] == subject_id]

    if subject_id == 105:
        val_data.append(subject_data)
    elif subject_id == 106:
        test_data.append(subject_data)
    else:
        train_data.append(subject_data)


In [ ]:
train_data = pd.concat(train_data)
val_data = pd.concat(val_data)
test_data = pd.concat(test_data)

In [ ]:
unique_train_ids = val_data['id'].unique()
print(unique_train_ids)

[105]


In [ ]:
def generate_sentence(df2):
    sentence = ""

    # Define the column names you want to extract values from
    columns_of_interest = ['heart_rate', 'hand_temperature', 'hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y',
                           'hand_3D_acceleration_16_z', 'hand_3D_acceleration_6_x', 'hand_3D_acceleration_6_y',
                           'hand_3D_acceleration_6_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y',
                           'hand_3D_gyroscope_z', 'hand_3D_magnetometer_x', 'hand_3D_magnetometer_y',
                           'hand_3D_magnetometer_z', 'chest_temperature', 'chest_3D_acceleration_16_x',
                           'chest_3D_acceleration_16_y', 'chest_3D_acceleration_16_z', 'chest_3D_acceleration_6_x',
                           'chest_3D_acceleration_6_y', 'chest_3D_acceleration_6_z', 'chest_3D_gyroscope_x',
                           'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z', 'chest_3D_magnetometer_x',
                           'chest_3D_magnetometer_y', 'chest_3D_magnetometer_z', 'ankle_temperature',
                           'ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y', 'ankle_3D_acceleration_16_z',
                           'ankle_3D_acceleration_6_x', 'ankle_3D_acceleration_6_y', 'ankle_3D_acceleration_6_z',
                           'ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z',
                           'ankle_3D_magnetometer_x', 'ankle_3D_magnetometer_y', 'ankle_3D_magnetometer_z']

    # Iterate over the columns of interest
    for column in columns_of_interest:
        # Get the value for the current column
        value = df2[column]  # Directly access the single value in the column

        # Add column name and value to the sentence
        sentence += "{}{}".format(column, value)

        # Add a space if it's not the last column
        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [ ]:
def sequencesToSentences(activitySequences):
    sentences = []
    label_sentences = []

    for _, row in activitySequences.iterrows():
        sentence = generate_sentence(row)
        sentences.append(sentence)
        label_sentences.append(row['activity_id'])

    return sentences, label_sentences

In [ ]:
train_sentences, train_label_sentences = sequencesToSentences(train_data)

In [ ]:
test_sentences, test_label_sentences = sequencesToSentences(test_data)

In [ ]:
val_sentences, val_label_sentences = sequencesToSentences(val_data)

In [ ]:
len(val_sentences)

272442

In [ ]:
train_sentences_array = np.array(train_sentences)
train_label_sentences_array = np.array(train_label_sentences)

# Define the file paths
sentences_file = "train_sentences.txt"
labels_file = "train_label_sentences.txt"

# Save the sentences and labels to separate text files
np.savetxt(sentences_file, train_sentences_array, fmt="%s")
np.savetxt(labels_file, train_label_sentences_array, fmt="%s")

In [ ]:
tokenizer1 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer1.fit_on_texts(train_sentences)
word_index1 = tokenizer1.word_index
train_indexed_sentences = tokenizer1.texts_to_sequences(train_sentences)

In [ ]:
tokenizer2 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer2.fit_on_texts(test_sentences)
word_index2 = tokenizer2.word_index
test_indexed_sentences = tokenizer2.texts_to_sequences(test_sentences)

In [ ]:
tokenizer3 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer3.fit_on_texts(val_sentences)
word_index3 = tokenizer3.word_index
val_indexed_sentences = tokenizer3.texts_to_sequences(val_sentences)

#### original

In [ ]:
def generate_sentence(df2):

    sentence = ""

    val = ""

    #extract sensors list
    sensors = df2.sensor.values

    values = df2.value.values

    #iterate on sensors list
    for i in range(len(sensors)):

        val = values[i]

        if i == len(sensors) - 1:
            sentence += "{}{}".format(sensors[i],val)
        else:
            sentence += "{}{} ".format(sensors[i],val)

    return sentence

#### with my columns

In [ ]:
def generate_sentence(df2):
    sentence = ""

    # Define the column names you want to extract values from
    columns_of_interest = ['heart_rate', 'hand_temperature', 'hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y',
                           'hand_3D_acceleration_16_z', 'hand_3D_acceleration_6_x', 'hand_3D_acceleration_6_y',
                           'hand_3D_acceleration_6_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y',
                           'hand_3D_gyroscope_z', 'hand_3D_magnetometer_x', 'hand_3D_magnetometer_y',
                           'hand_3D_magnetometer_z', 'chest_temperature', 'chest_3D_acceleration_16_x',
                           'chest_3D_acceleration_16_y', 'chest_3D_acceleration_16_z', 'chest_3D_acceleration_6_x',
                           'chest_3D_acceleration_6_y', 'chest_3D_acceleration_6_z', 'chest_3D_gyroscope_x',
                           'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z', 'chest_3D_magnetometer_x',
                           'chest_3D_magnetometer_y', 'chest_3D_magnetometer_z', 'ankle_temperature',
                           'ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y', 'ankle_3D_acceleration_16_z',
                           'ankle_3D_acceleration_6_x', 'ankle_3D_acceleration_6_y', 'ankle_3D_acceleration_6_z',
                           'ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z',
                           'ankle_3D_magnetometer_x', 'ankle_3D_magnetometer_y', 'ankle_3D_magnetometer_z']

    # Iterate over the columns of interest
    for column in columns_of_interest:
        # Get the value for the current column
        value = df2[column].values[0]  # Directly access the single value in the column

        # Add column name and value to the sentence
        sentence += "{}{}".format(column, value)

        # Add a space if it's not the last column
        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [ ]:
def sequencesToSentences(activitySequences):
	sentences = []
	label_sentences = []

	for i in range(len(activitySequences)):

		sentence = generate_sentence(activitySequences[i])

		sentences.append(sentence)
		label_sentences.append(activitySequences[i].activity_id.values[0])

	return sentences, label_sentences

In [ ]:
sentences, label_sentences = sequencesToSentences(activitySequences)

sentences indexization

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
indexed_sentences = tokenizer.texts_to_sequences(sentences)

In [ ]:
len(tokenizer.word_index)

4038

In [ ]:
indexed_sentences

sliding windows

In [ ]:
def slidingWindow(sequence,winSize,step=1):

    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")

    numOfChunks = int(((len(sequence)-winSize)/step)+1)

    # Do the work
    if winSize > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i in range(0,numOfChunks*step,step):
            yield sequence[i:i+winSize]

In [ ]:
X_windowed = []
Y_windowed = []
winSize = 50
step = 1

for i,s in enumerate(indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_windowed.append(chunk)
		Y_windowed.append(label_sentences[i])

In [ ]:
padded_windows = pad_sequences(X_windowed)

In [ ]:
Y_windowed = np.array(Y_windowed)

## Save files ##
print("STEP 8: save sliding windows and labels")
np.save("{}_{}_padded_x.npy".format("lstm",winSize), padded_windows)
np.save("{}_{}_padded_y.npy".format("lstm",winSize), Y_windowed)

STEP 8: save sliding windows and labels


In [ ]:
padded_windows.shape

(14593, 50)

In [ ]:
padded_windows[0].shape

(50,)

### main

In [19]:
train_padded_windows = np.load("{}_{}_tpadded_x.npy".format("lstm", winSize))
Y_train_windowed = np.load("{}_{}_tpadded_y.npy".format("lstm", winSize))
test_padded_windows = np.load("{}_{}_ttpadded_x.npy".format("lstm", winSize))
Y_test_windowed = np.load("{}_{}_ttpadded_y.npy".format("lstm", winSize))
val_padded_windows = np.load("{}_{}_vpadded_x.npy".format("lstm", winSize))
Y_val_windowed = np.load("{}_{}_vpadded_y.npy".format("lstm", winSize))

In [20]:
Y_train_windowed.shape

(2095,)

In [21]:
x_train = train_padded_windows
y_train = Y_train_windowed
x_test = test_padded_windows
y_test = Y_test_windowed
x_val = val_padded_windows
y_val = Y_val_windowed

#### FCN embedded

In [22]:
pip install tensorflow

In [ ]:
class LSTMModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(hidden_dim)
        self.fc = tf.keras.layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
        embedded = self.embedding(inputs)
        lstm_out = self.lstm(embedded)
        output = self.fc(lstm_out)
        return output

In [ ]:
flat_x_train = [token for sublist in x_train for token in sublist]
vocab_size = len(set(flat_x_train))
embedding_dim = 100
hidden_dim = 128
output_dim = 25

In [ ]:
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 for the padding token if used

print("Vocabulary size:", vocab_size)

Vocabulary size: 9747


In [ ]:
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=20, validation_data=(x_val, y_val))

105/105 [==============================] - 21s 203ms/step - loss: 1.4074 - accuracy: 0.4339 - val_loss: 2.7134 - val_accuracy: 0.2074


#### predict

In [ ]:
predictions = model.predict(x_test)

366/366 [==============================] - 28s 76ms/step


In [ ]:
predictions.shape

In [ ]:
import numpy as np

class_predictions = np.argmax(predictions, axis=1)

In [ ]:
class_predictions

array([16,  6,  3, ...,  2, 24,  6])

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, class_predictions)
print("Accuracy:", accuracy)

Accuracy: 0.2434418525164488


#### try a more complex lstm

In [ ]:
class ImprovedLSTMModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(ImprovedLSTMModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm1 = tf.keras.layers.LSTM(hidden_dim, return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(0.5)  # Example dropout rate
        self.lstm2 = tf.keras.layers.LSTM(hidden_dim)
        self.fc = tf.keras.layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
        embedded = self.embedding(inputs)
        lstm_out1 = self.lstm1(embedded)
        lstm_out2 = self.lstm2(self.dropout(lstm_out1))
        output = self.fc(lstm_out2)
        return output

In [ ]:
model = ImprovedLSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=20, validation_data=(x_val, y_val))

Epoch 1/3
105/105 [==============================] - 42s 358ms/step - loss: 1.9616 - accuracy: 0.1776 - val_loss: 1.7154 - val_accuracy: 0.1778
Epoch 2/3
105/105 [==============================] - 56s 536ms/step - loss: 1.8428 - accuracy: 0.1933 - val_loss: 1.6910 - val_accuracy: 0.2081
Epoch 3/3
105/105 [==============================] - 56s 536ms/step - loss: 1.8326 - accuracy: 0.1905 - val_loss: 1.6907 - val_accuracy: 0.2456


trying another model

In [24]:
import torch.nn as nn

def make_list(val):
        if type(val) is int:
            return (val, val)
        elif type(val) is tuple and len(val) == 2:
            return val
        else:
            raise TypeError("Only int and tuples allowed")

class BasicConv2d(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride = 1, padding = 0):
        super(BasicConv2d, self).__init__()

        side_pad, top_pad = make_list(padding)
        side_ker, top_ker = make_list(kernel_size)
        side_stride, top_stride = make_list(stride)


        self.conv = nn.Sequential(nn.Conv2d(in_planes, in_planes,
                                            kernel_size=1, bias=True),
                                  nn.Conv2d(in_planes, in_planes,
                                            kernel_size=(side_ker,1), stride=(side_stride,1),
                                            padding=(side_pad,0), bias=False),
                                  nn.Conv2d(in_planes, in_planes,
                                            kernel_size=(1,top_ker), stride=(1,top_stride),
                                            padding=(0,top_pad), bias=False),
                                  nn.Conv2d(in_planes, out_planes,
                                            kernel_size=1, bias=True),
                                 )
        self.bn = nn.BatchNorm2d(out_planes,
                                 eps=0.001,
                                 momentum=0.1,
                                 affine=True)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [25]:
class DeepConvNet(nn.Module):

    def __init__(self, in_channels = 3, input_size = 100, hidden_size = 128, output_size = 7, conv_filter = (5,9), conv_padding = (2,4)):
        super(DeepConvNet,self).__init__()
        self.in_channels = in_channels
        self.input_size = input_size
        ## Input : 3 * 9 * 100 for PAMAP2
        self.conv = nn.Sequential(BasicConv2d(in_channels,64,conv_filter,padding = conv_padding),
                                  ## 64 * 9 * 100
                                 BasicConv2d(64,256,conv_filter,padding = conv_padding),
                                  ## 256 * 9 * 100
                                 BasicConv2d(256,512,conv_filter,padding = conv_padding),
                                  ## 512 * 9 * 100
                                 nn.AdaptiveAvgPool2d((10,10))
                                  ## 512 * 10 * 10
                                 )
        ## Input to LSTM : 512 * 100
        self.lstm = nn.LSTM(input_size = 100, hidden_size = hidden_size, num_layers = 2, batch_first = True)
        ## 512 * 128(hidden_size)
        self.fcn = nn.Sequential(nn.Flatten(),nn.ReLU(),nn.Dropout(p=0.2),
                                 nn.Linear(512*hidden_size, output_size))
        ## 7(output_size)

    def forward(self, input_seq):
        input_seq = input_seq.view(input_seq.shape[0],self.in_channels,-1,self.input_size)
        y = self.conv(input_seq)
        y, h = self.lstm(y.view(y.shape[0],-1,100))
        y = self.fcn(y)
        return y